In [1]:
import torch
import sys
sys.path.append('..')

In [3]:
import numpy as np
import sys
import math
import os
import torch
from tqdm import tqdm
from tensorboardX import SummaryWriter

import copy

from models import LinearModel, convert_vars_to_gpu
from utils import logsumexp, shuffle_combined, torchify, exp_lr_scheduler, get_optimizer
from datareader import reader_from_pickle, DataReader
from args import args
from evaluation import sample_ranking, compute_dcg_rankings, compute_dcg, compute_average_rank, sample_multiple_ranking
from fairness_loss import get_exposures, get_multiple_exposures, GroupFairnessLoss

In [7]:
trdr, vadr, tedr = reader_from_pickle(
    "../GermanCredit/MSLR_Fold1/full/train.pkl"), reader_from_pickle(
    "../GermanCredit/MSLR_Fold1/full/valid.pkl"), reader_from_pickle(
    "../GermanCredit/MSLR_Fold1/full/test.pkl")
tr,va, te = list(trdr.data), list(vadr.data), list(tedr.data)
len(tr[0]), len(va[0]), len(te[0])

(11974, 4009, 4065)

In [3]:
probs=torch.tensor([0.1000, 0.1000, 0.7000, 0.1000, 0.1000, 0.9000, 0.1000, 0.1000, 0.1000,
         0.1000])
position_bias_vector = 1/torch.arange(1.,11.)
rankings = sample_multiple_ranking(probs, 5)

In [4]:
get_multiple_exposures(rankings.cuda(), 1/torch.arange(1.,11.).cuda())

tensor([[0.1667, 0.2000, 1.0000, 0.5000, 0.1111, 0.3333, 0.1250, 0.2500, 0.1429,
         0.1000],
        [0.3333, 0.2000, 0.2500, 0.1429, 0.5000, 1.0000, 0.1000, 0.1111, 0.1667,
         0.1250],
        [0.1111, 0.1429, 1.0000, 0.2500, 0.1667, 0.5000, 0.1000, 0.2000, 0.1250,
         0.3333],
        [0.2000, 0.5000, 0.3333, 0.1111, 0.1250, 1.0000, 0.1429, 0.1000, 0.1667,
         0.2500],
        [0.3333, 0.1250, 1.0000, 0.1667, 0.1111, 0.5000, 0.1429, 0.1000, 0.2500,
         0.2000]], device='cuda:0')

In [5]:
for i in range(rankings.shape[0]):
    print(get_exposures(rankings[i], 1/torch.arange(1.,11.)))

tensor([0.1667, 0.2000, 1.0000, 0.5000, 0.1111, 0.3333, 0.1250, 0.2500, 0.1429,
        0.1000])
tensor([0.3333, 0.2000, 0.2500, 0.1429, 0.5000, 1.0000, 0.1000, 0.1111, 0.1667,
        0.1250])
tensor([0.1111, 0.1429, 1.0000, 0.2500, 0.1667, 0.5000, 0.1000, 0.2000, 0.1250,
        0.3333])
tensor([0.2000, 0.5000, 0.3333, 0.1111, 0.1250, 1.0000, 0.1429, 0.1000, 0.1667,
        0.2500])
tensor([0.3333, 0.1250, 1.0000, 0.1667, 0.1111, 0.5000, 0.1429, 0.1000, 0.2500,
        0.2000])


In [6]:
dr = reader_from_pickle("../GermanCredit/partial_german_train_rank_weightedclick_10k.pkl")
feats,rels = dr.data
feats,rels = torch.FloatTensor(feats), torch.FloatTensor(rels)
f,r = feats[41], rels[41]
f,r = torch.FloatTensor(f), torch.FloatTensor(r)

group_identities = f[:,13]
a = 1 if r[group_identities == 0].mean() >= r[group_identities == 1].mean() else -1

In [7]:
group_identities == 0, r
rel = r

In [8]:
torch.cuda.FloatTensor([0.])

tensor([0.], device='cuda:0')

In [9]:
GroupFairnessLoss.compute_multiple_group_disparity(rankings.cuda(), rel, group_identities, 1/torch.arange(1.,11.).cuda())

tensor([0., 0., 0., 0., 0.], device='cuda:0')

In [10]:
for i in range(rankings.shape[0]):
    print(GroupFairnessLoss.compute_group_disparity(rankings[i], rel, group_identities, 1/torch.arange(1.,11.)))

tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)


In [147]:
from models import LinearModel
from torch.autograd import Variable
model = LinearModel(D=58).cuda()

In [151]:
train_feats= feats
train_rel = rels
sample_size = 10
position_bias_vector = (1. / torch.arange(1., 100.)).cuda()
len_train_set=len(train_feats)
    
group_disparity_indicator_batch_size = int(len_train_set/20)
group_disp_feats = train_feats[torch.randperm(
    train_feats.shape[0])[:group_disparity_indicator_batch_size]].cuda()
group_disp_rels = train_rel[torch.randperm(
    train_feats.shape[0])[:group_disparity_indicator_batch_size]].cuda()
scores = model(Variable(group_disp_feats)).squeeze()
probs = torch.nn.Softmax(dim=0)(scores)
indicator_disparities = []

for i in range(group_disparity_indicator_batch_size):
    rankings = sample_multiple_ranking(probs[i], sample_size)
    group_identities = group_disp_feats[i][:, args.group_feat_id]
    indicator_disparities.append(GroupFairnessLoss.compute_multiple_group_disparity(
    rankings, group_disp_rels[i], group_identities, position_bias_vector).mean())
indicator_disparities = torch.stack(indicator_disparities)
indicator_disparities.mean()

tensor(0.1207, device='cuda:0')

In [132]:
indicator_disparities[0]
indicator_disparities = torch.cat((indicator_disparities[450:], indicator_disparities[0].view(1)))
indicator_disparities

tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.5072, -3.2102,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000, -1.2832,  0.0000,  0.0000,  0.0000,  0.0000,
         3.9912,  1.0180,  0.0000,  0.0000,  0.6887,  4.5148,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  4.8611,  0.3627,  1.5239,
         0.0000,  0.0000,  0.3880,  0.3885,  0.0000,  0.0000,  1.0787,  0.0000,
         3.0205,  0.0000,  0.0000,  0.6635,  0.0000,  0.0000,  0.0000,  2.4734,
         0.0000,  0.5244, -1.2772], device='cuda:0')

In [101]:
indicator_disparities[9].view(1)

tensor([1.1210], device='cuda:0')

In [164]:
torch.sort(-probs[0])[1].unsqueeze(0).shape

torch.Size([1, 10])